In [41]:
import sys
from joblib import dump,load

In [42]:
sys.path.append('../src/data/')
sys.path.append('../src/models/')

In [43]:
from data_preparation import DataPreparation
from train_model import TrainModel
from predict_model import PredictModel
import pandas as pd

In [44]:
def run_random_forest(X, y, test_size=0.3, random_state=42):
    train_model_rf = TrainModel(X, y, test_size=test_size, random_state=random_state)
    rf_model, X_val_rf, y_val_rf = train_model_rf.train_random_forest()

    predict_model_rf = PredictModel(rf_model, X_val_rf, y_val_rf)
    val_auroc_rf = predict_model_rf.evaluate_model()
    
    print(f"Random Forest Validation AUROC: {val_auroc_rf}")
    
    return predict_model_rf, train_model_rf# Return the PredictModel instance

In [45]:
def run_xgboost(X, y, test_size=0.3, random_state=42):
    train_model_xgb = TrainModel(X, y, test_size=test_size, random_state=random_state)
    xgb_model, X_val_xgb, y_val_xgb = train_model_xgb.train_xgboost()

    predict_model_xgb = PredictModel(xgb_model, X_val_xgb, y_val_xgb)
    val_auroc_xgb = predict_model_xgb.evaluate_model()
    
    print(f"XGBoost Validation AUROC: {val_auroc_xgb}")
    
    return predict_model_xgb, train_model_xgb # Return the PredictModel instance

In [46]:
def create_submission(predict_model, X_test, test_player_ids, filename):
    predictions = predict_model.predict_on_test(X_test)
    submission = pd.DataFrame({
        'player_id': test_player_ids,
        'drafted': predictions
    })
    submission.to_csv(filename, index=False)

In [47]:
if __name__ == "__main__":
    data_prep = DataPreparation('/Users/shivatmaksharma/D/Uni Work/AdvMl_Project/data/raw/train.csv', 
                                '/Users/shivatmaksharma/D/Uni Work/AdvMl_Project/data/raw/test.csv')
    
    data_prep.preprocess_data()
    X, y, test_player_ids, X_test = data_prep.extract_features_and_target()

    predict_model_rf, train_model_rf = run_random_forest(X, y)
    create_submission(predict_model_rf, X_test, test_player_ids, '/Users/shivatmaksharma/D/Uni Work/rf_submission.csv')

    predict_model_xgb, train_model_xgb = run_xgboost(X, y)
    create_submission(predict_model_xgb, X_test, test_player_ids, '/Users/shivatmaksharma/D/Uni Work/xgb_submission.csv')



/Users/shivatmaksharma/D/Uni Work/AdvMl_Project/notebooks/../src/data/data_preparation.py:78: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.test_data[col] = 0
/Users/shivatmaksharma/D/Uni Work/AdvMl_Project/notebooks/../src/data/data_preparation.py:78: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.test_data[col] = 0
/Users/shivatmaksharma/D/Uni Work/AdvMl_Project/notebooks/../src/data/data_preparation.py:78: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

Random Forest Validation AUROC: 0.9922351863521737
XGBoost Validation AUROC: 0.9970887538770963


In [48]:
dump(train_model_rf, '../models/rf_model.joblib')
dump(train_model_xgb, '../models/xgb_model.joblib')

['../models/xgb_model.joblib']